In [320]:
from pandas import read_csv, DataFrame, Index

In [321]:
huobi_data = read_csv('data/btc inverse perps/Huobi_1d.csv', parse_dates=['timestamp'], index_col='timestamp')
huobi_data.name = 'Huobi'

binance_data = read_csv('data/btc inverse perps/Binance_1d.csv', parse_dates=['timestamp'], index_col='timestamp')
binance_data.name = 'Binance'

deribit_data = read_csv('data/btc inverse perps/Deribit_1d.csv', parse_dates=['timestamp'], index_col='timestamp')
deribit_data.name = 'Deribit'

bybit_data = read_csv('data/btc inverse perps/Bybit_1d.csv', parse_dates=['timestamp'], index_col='timestamp')
bybit_data.name = 'Bybit'

bitget_data = read_csv('data/btc inverse perps/Bitget_1d.csv', parse_dates=['timestamp'], index_col='timestamp')
bitget_data.name = 'Bitget'

bitmex_data = read_csv('data/btc inverse perps/Bitmex_1d.csv', parse_dates=['timestamp'], index_col='timestamp')
bitmex_data.name = 'Bitmex'

okx_data = read_csv('data/btc inverse perps/OKX_1d.csv', parse_dates=['timestamp'], index_col='timestamp')
okx_data.name = 'OKX'

kraken_data = read_csv('data/btc inverse perps/Kraken_1d.csv', parse_dates=['timestamp'], index_col='timestamp')
kraken_data.name = 'Kraken'

In [322]:
dfs_2020Q2_2023Q1 = [huobi_data, deribit_data, bybit_data, bitmex_data, okx_data]
dfs_2023Q2 = [huobi_data, deribit_data, bybit_data, bitmex_data, okx_data, bitget_data, kraken_data]
dfs_2023Q3_2023Q4 = [huobi_data, deribit_data, bybit_data, bitmex_data, okx_data, binance_data, bitget_data, kraken_data]

In [323]:
def agg_OI_quarterly(df):
    ret = df['OI Close'].resample('Q').sum()
    ret = ret.to_frame(name='Agg OI').reset_index()
    ret['Quarter'] = ret['timestamp'].dt.to_period('Q')
    ret.name = df.name
    return ret

In [324]:
dfs_2020Q2_2023Q1_quarterly_OI = [agg_OI_quarterly(df) for df in dfs_2020Q2_2023Q1]
dfs_2023Q2_quarterly_OI = [agg_OI_quarterly(df) for df in dfs_2023Q2]
dfs_2023Q3_2023Q4 = [agg_OI_quarterly(df) for df in dfs_2023Q3_2023Q4]

In [325]:
def get_ratio(df, dfs, quarter):
    sum_of_quarter = 0
    this_value = df.loc[df['Quarter'] == quarter]['Agg OI'].values[0]
    for _df in dfs:
        sum_of_quarter += _df.loc[_df['Quarter'] == quarter]['Agg OI'].values[0]
    return df.name, quarter, this_value / sum_of_quarter

In [326]:
quarter_list = [
    '2020Q2', '2020Q3', '2020Q4', '2021Q1', '2021Q2', '2021Q3', '2021Q4', '2022Q1',
    '2022Q2', '2022Q3', '2022Q4', '2023Q1', '2023Q2', '2023Q3', '2023Q4'
]

In [327]:
ratios_by_quarter = []

for quarter in quarter_list:
    ratios = []

    if quarter == '2023Q2':
        for df in dfs_2023Q2_quarterly_OI:
            _, _, ratio = get_ratio(df, dfs_2023Q2_quarterly_OI, quarter)
            ratios.append(ratio)
    elif quarter in ['2023Q3', '2023Q4']:
        for df in dfs_2023Q3_2023Q4:
            _, _, ratio = get_ratio(df, dfs_2023Q3_2023Q4, quarter)
            ratios.append(ratio)
    else:
        for df in dfs_2020Q2_2023Q1_quarterly_OI:
            _, _, ratio = get_ratio(df, dfs_2020Q2_2023Q1_quarterly_OI, quarter)
            ratios.append(ratio)

    ratios_by_quarter.append(ratios)

In [328]:
for i in range(len(ratios_by_quarter)):
    if i <= 11:
        ratios_by_quarter[i].insert(5, 0)
        ratios_by_quarter[i].insert(6, 0)
        ratios_by_quarter[i].insert(7, 0)
    if i == 12:
        ratios_by_quarter[i].insert(5, 0)

In [329]:
oi_ratios_by_quarter_and_exchange = DataFrame(
    ratios_by_quarter,
    columns=['Huobi', 'Deribit', 'Bybit', 'Bitmex', 'OKX', 'Binance', 'Biteget', 'Kraken']
)

oi_ratios_by_quarter_and_exchange = oi_ratios_by_quarter_and_exchange.set_index(Index(quarter_list))
oi_ratios_by_quarter_and_exchange['Checksum'] = oi_ratios_by_quarter_and_exchange.sum(axis=1)

In [330]:
oi_ratios_by_quarter_and_exchange

,Huobi,Deribit,Bybit,Bitmex,OKX,Binance,Biteget,Kraken,Checksum
2020Q2,0.091932,0.074491,0.171595,0.550159,0.111822,0.000000,0.000000,0.000000,1.0
2020Q3,0.153404,0.072746,0.213648,0.467397,0.092805,0.000000,0.000000,0.000000,1.0
2020Q4,0.144935,0.111903,0.361464,0.274520,0.107177,0.000000,0.000000,0.000000,1.0
2021Q1,0.128307,0.139291,0.455804,0.189173,0.087424,0.000000,0.000000,0.000000,1.0
2021Q2,0.125797,0.153031,0.455724,0.175106,0.090341,0.000000,0.000000,0.000000,1.0
2021Q3,0.091755,0.180789,0.413364,0.228944,0.085148,0.000000,0.000000,0.000000,1.0
2021Q4,0.047300,0.193181,0.484304,0.157884,0.117330,0.000000,0.000000,0.000000,1.0
2022Q1,0.048794,0.237229,0.373900,0.187355,0.152722,0.000000,0.000000,0.000000,1.0
2022Q2,0.036651,0.260827,0.323576,0.164361,0.214584,0.000000,0.000000,0.000000,1.0
2022Q3,0.032399,0.263301,0.268561,0.175353,0.260385,0.000000,0.000000,0.000000,1.0
